In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import GridSearchCV

train_data_path = '/home/gram/25-2-ML-final-report/data/train_processed.csv'
test_data_path = '/home/gram/25-2-ML-final-report/data/test_processed.csv'

train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

train_df.head()

,Age,Annual Income,Number of Dependents,Education Level,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,...,Occupation_Self-Employed,Occupation_Unemployed,Occupation_Unknown,Location_Suburban,Location_Urban,Policy Type_Comprehensive,Policy Type_Premium,Smoking Status_Yes,Property Type_Condo,Property Type_House
0,19.0,10049.0,1.0,1,22.598761,2.0,17.0,372.00000,5.0,0,...,True,False,False,False,True,False,True,False,False,True
1,39.0,31678.0,3.0,2,15.569731,1.0,12.0,694.00000,2.0,1,...,False,False,True,False,False,True,False,True,False,True
2,23.0,25602.0,3.0,0,47.177549,1.0,14.0,592.92435,3.0,2,...,True,False,False,True,False,False,True,True,False,True
3,21.0,141855.0,2.0,1,10.938144,1.0,0.0,367.00000,1.0,0,...,False,False,True,False,False,False,False,True,False,False
4,21.0,39651.0,1.0,1,20.376094,0.0,8.0,598.00000,4.0,0,...,True,False,False,False,False,False,True,True,False,True


In [2]:
# 훈련데이터에서 검증데이터셋 분리
from sklearn.model_selection import train_test_split

X = train_df.drop('Premium Amount', axis=1)
y = train_df['Premium Amount']

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)


In [3]:
# 객체 생성 및 모델 훈련
model = XGBRegressor()
model.fit(X_train,y_train, eval_set=[(X_val, y_val)])

[0]	validation_0-rmse:857.10074
[1]	validation_0-rmse:853.32684
[2]	validation_0-rmse:851.54320
[3]	validation_0-rmse:849.16268
[4]	validation_0-rmse:848.47511
[5]	validation_0-rmse:848.08878
[6]	validation_0-rmse:847.72525
[7]	validation_0-rmse:847.19003
[8]	validation_0-rmse:847.15092
[9]	validation_0-rmse:847.09095
[10]	validation_0-rmse:846.96590
[11]	validation_0-rmse:846.72852
[12]	validation_0-rmse:846.66446
[13]	validation_0-rmse:846.54482
[14]	validation_0-rmse:846.48903
[15]	validation_0-rmse:846.29852
[16]	validation_0-rmse:846.15455
[17]	validation_0-rmse:846.16157
[18]	validation_0-rmse:846.12873
[19]	validation_0-rmse:845.56203
[20]	validation_0-rmse:845.57065
[21]	validation_0-rmse:845.40322
[22]	validation_0-rmse:845.40631
[23]	validation_0-rmse:845.28955
[24]	validation_0-rmse:845.29616
[25]	validation_0-rmse:845.27198
[26]	validation_0-rmse:845.28358
[27]	validation_0-rmse:845.26819
[28]	validation_0-rmse:845.29984
[29]	validation_0-rmse:845.07619
[30]	validation_0-rm

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [ ]:
y_pred = model.predict(X_val)
mae = mean_absolute_error(y_val,y_pred)
rmse = root_mean_squared_error(y_val, y_pred)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

MAE: 641.7947262900989
RMSE: 844.7643298076317


: 

In [ ]:
# 예시: 조합 개수 = 3 x 2 x 2 = 12번 (여기에 CV 횟수만큼 곱해짐)
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [5, 7]
}

xgb_model = XGBRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error', # 평가는 MAE로 (sklearn은 오차가 작을수록 좋다는 걸 음수로 표현합니다)
    cv=3,       # 3-Fold 교차 검증 (데이터를 3등분해서 3번 돌려봄)
    verbose=2,  # 진행 상황 출력
    n_jobs=-1   # 병렬 처리로 속도 업
)

grid_search.fit(X_train, y_train)

# 최고의 파라미터와 점수 확인
print("Best Params:", grid_search.best_params_)
print("Best Score:", -grid_search.best_score_) # 음수로 나오므로 다시 - 붙여줌

Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [ ]:
best_model = grid_search.best_estimator_